In [ ]:
import music21
import numpy as np
import matplotlib.pyplot as plt
import copy

In [ ]:
# load bach chorales
# this search takes a moment, avoid re-runnign this cell
chorales = music21.corpus.search('bach', fileExtensions='xml')

In [ ]:
'''
Problem 1. (required). Load a Bach Chorale using music 21 and 
the compute the number of occurances of each musical interval
occuring in horizontal motion within each vocal part in the piece you choose.
Up to you to choose how to report it (e.g. does 3rd equal 10th)
Here are some examples of\music21 code/objects to get you started.
'''
bwv1 = chorales[0].parse()
soprano_part = bwv1.parts[0]
# these are Pitch objects
soprano_pitches = soprano_part.pitches
a_single_pitch_object = soprano_pitches[0]
# numerical interval
print(a_single_pitch_object.ps)

# some other useful music21 things
Note = music21.note.Note
Duration = music21.duration.Duration
Stream = music21.stream.Stream
Pitch = music21.pitch.Pitch
Interval = music21.interval.Interval

n1 = Note(60.0)
n1.duration = Duration(1.0)
n2 = Note(65.0)
int_n1_n2 = Interval(n1,n2)
print(int_n1_n2)
n1.pitch.ps = n1.pitch.ps - 2
int_n1_n2 = Interval(n1,n2)
print(int_n1_n2)

In [ ]:
'''
Problem 2 (optional, for your exploration)

Programmatically generate a few measures of score in music21,
and experiment with durations, articulations, ties, and beaming.
I give some examples below.
'''

Tie = music21.tie.Tie
Measure = music21.stream.Measure
Articulation = music21.articulations.Articulation

def demo_tie():
    # handling Ties
    c0 = Note('C4')
    c0.tie = Tie('start')
    c1 = Note('C4')
    c1.tie = Tie('continue')
    c1.tie.placement = 'above'
    c2 = Note('C4')
    c2.tie = Tie('stop')
    s = music21.stream.Measure()
    s.append([c0, c1, c2])
    s.show()
    
def demo_beam():
    beat1 = Note('C4', type='quarter')
    beat2 = [Note('D4', type='eighth') for i in range(2)]
    beat3 = [Note('E4', type='16th') for i in range(4)]
    m1 = Measure()    
    m1.append(beat1)
    m1.append(beat2)
    m1.append(beat3)
    m2 = copy.deepcopy(m1)
    s=Stream()
    s.append([m1,m2])
    s.show()
    
def demo_articulation():
    art = Articulation
    art.placement = 'above'
    
    stac = music21.articulations.Staccato()
    acc = music21.articulations.Accent()
    
    n1 = Note('A4')
    print(n1.articulations)
    n1.articulations.append(stac)
    print(n1.articulations)
    n1.show()
    
    n2 = Note('A4')
    sacc = music21.articulations.StrongAccent()
    n2.articulations.append(sacc) 
    sacc2 = music21.articulations.StrongAccent()
    sacc2.pointDirection = 'down'
    sacc2.placement = 'above'
    n3 = Note('A4')
    n3.articulations.append(sacc2)
    m = Measure()
    m.append([n2, n3]) 
    m.show()

    n12, n22 = n1.splitAtQuarterLength(0.5)
    m2 = Measure()
    m2.append([n12, n22])
    m2.show()

    n1.show()
    doit = music21.articulations.Doit()
    n1.articulations.append(doit)
    n1.show()


demo_tie()
demo_beam()
demo_articulation()

In [ ]:
import librosa
import librosa.display as librosa_display
import IPython.display as ipd

In [ ]:
'''
Problem 3. (required).

In the following, I review how to:
- load a clip
- visualize its waveplot
- visuzlize its STFT
- use some onset detection code
- plot with onsets
- listen to audio with clicks at onsets

Then, I define the function 'get_onsets_fancy'.
This function allows you to specify a threshold
for onset strength for what should register as an onset.

Your task is to 
- try onset detection on a clip of your choice
- try to improve its performance 
  by experimenting with the threshold.
  e.g. lower threshold if some attacks are missed.
  
Step 1. Replace the trumpet sample with a small audio 
recording of your choice. You should test something with clear onsets of musical events.

Step 2. experiment with threshold
'''

In [ ]:
# some helper functions
def load_audio(path=None):
    if path is None:
        path = librosa.ex('trumpet')
    signal, sample_rate = librosa.load(path)
    return signal, sample_rate

def compute_log_spec(x):
    S = librosa.stft(x)
    absS = np.abs(S)
    logS = np.log(absS)
    return logS

def get_onsets(x,sr):
    onset_frames = librosa.onset.onset_detect(x, sr)
    onset_times = librosa.frames_to_time(onset_frames, sr)
    return onset_times, onset_frames

In [ ]:
x,sr = load_audio()
print("first few samples of signal",x[:3])
print("sample rate",sr)

onset_times, onset_frames = get_onsets(x,sr)

logS = compute_log_spec(x)
NUM_FREQS = logS.shape[0]
NUM_TIMEWINDOWS = logS.shape[1]

# waveplot with onset time lines
librosa_display.waveplot(x, sr)
plt.vlines(onset_times, -0.8, 0.79, color='r', alpha=0.8)

In [ ]:
# log spec with onset time lines. log spec y axis is in log scale.
librosa_display.specshow(logS, sr=sr, x_axis='time',y_axis='log')
plt.vlines(onset_times, 0, logS.shape[0], color='k', alpha=0.8)

In [ ]:
# make a click track
clicks = librosa.clicks(frames=onset_frames, sr=sr, length=len(x))
ipd.Audio(x + clicks,rate=sr)

In [ ]:
def get_onsets_fancy(x, sr):
    onset_frames = librosa.onset.onset_detect(x, sr=sr)
    all_strengths = librosa.onset.onset_strength(x,sr=sr)
    onset_strengths = all_strengths[onset_frames]
    frames_strengths = zip(onset_frames,onset_strengths)
    mean = np.mean(onset_strengths)
    variance = np.var(onset_strengths)
    '''
    keep frame if strength as at least threshold
    threshold is a little below the average strength.
    '''
    threshold = mean - (.4*variance)
    
    filtered = []
    for frame,strength in zip(onset_frames,onset_strengths):
        if strength > threshhold:
            filtered.append(frame)
    
    frames = filtered
    times = librosa.frames_to_time(frames, sr)
    return times, frames

In [ ]:
# listen to click track of onsets detected
onset_times, onset_frames = get_onsets_fancy(x,sr)
clicks = librosa.clicks(frames=onset_frames, sr=sr, length=len(x))
ipd.Audio(x + clicks,rate=sr)